In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Data Exploration
- Original Data Source: <a>https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#</a>
- Pandas Ready (not cleaned but ready as you can see in step 1) <a>https://github.com/TrainingByPackt/Data-Science-Projects-with-Python/tree/master/Data</a>
Set info: 

This research aimed at the case of customers default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods. From the perspective of risk management, the result of predictive accuracy of the estimated probability of default will be more valuable than the binary result of classification - credible or not credible clients. Because the real probability of default is unknown, this study presented the novel a Sorting Smoothing Method to estimate the real probability of default. 

With the real probability of default as the response variable (Y), and the predictive probability of default as the independent variable (X), the simple linear regression result (Y = A + BX) shows that the forecasting model produced by artificial neural network has the highest coefficient of determination; its regression intercept (A) is close to zero, and regression coefficient (B) to one. Therefore, among the six data mining techniques, artificial neural network is the only one that can accurately estimate the real probability of default.

#### Set Attribute Information:
This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. 

This study reviewed the literature and used the following 23 variables as explanatory variables:

Limit_Bal: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.

Sex: Gender (1 = male; 2 = female).

Education: (1 = graduate school; 2 = university; 3 = high school; 4 = others).

Marriage: (1 = married; 2 = single; 3 = others).

Age: (year).


<b>Note: X6-11 are labeled PAY_1-PAY_6</b>

X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: 

X6 = the repayment status in September, 2005; 

X7 = the repayment status in August, 2005; . . .;

X11 = the repayment status in April, 2005. 


<b>The measurement scale for the repayment status is:</b>

- -1 = pay duly; 

- 1 = payment delay for one month; 

- 2 = payment delay for two months; . . .;

- 8 = payment delay for eight months; 

- 9 = payment delay for nine months and above.



<b>Note: X12-17 are labeled BILL_AMT1-BILL_AMT6</b>

X12-X17: Amount of bill statement (NT dollar). 


<b>Note: X18-23 are labeled PAY_AMT1-PAY_AMT6</b>

<p> PAY1-6 are a record of past payments. Past monthly payments recorded from April to September.</P>

X18-X23: Amount of previous payment (NT dollar). 

X18 = amount paid in September, 2005; 

X19 = amount paid in August, 2005; . . .;

X23 = amount paid in April, 2005.

### The Business Problem:

The client is a credit card company that has brought forth some data for demographics and financials over the past 6 months for a sample of 30k account holders. The data is at the credit account level; in other words, there is one row for each account. Rows are defined whether in the next month after the six month historical data period, the account owner has defaulted or in other words, failed to make the minimum payment. 

#### Objective:
- Develop a predicitve model for whether an account will default next month given given the demographics and historical data. 

In [6]:
# Step 1 - Read in the data set
df = pd.read_excel('../data/default_of_credit_card_clients__courseware_version_1_21_19.xlsx')

C:\Users\Admin\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [7]:
# view the data
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [8]:
# get a unique count of ID's
df['ID'].nunique()

29687

In [9]:
# get an overall row count and shape
df.shape

(30000, 25)

In [10]:
# Obviously, there are some dups that need to be explored when there are 30k rows and 29687 unique ID'S. 
# Use value_counts to explore.
id_counts = df['ID'].value_counts()
id_counts.head()

87dec940-75b7    2
379e0b03-4ca0    2
c8bb762f-3ef9    2
c89b26ff-ec7f    2
2a793ecf-05c6    2
Name: ID, dtype: int64

In [11]:
# split it up to see how many rows have more than one count, simlar to a group by in SQL, group those > 1. We see 313 of them!
id_counts.value_counts()

1    29374
2      313
Name: ID, dtype: int64

#### Create a boolean mask to help clean the dups.

In [14]:
dupe_mask = id_counts == 2

dupe_mask[0:5]

87dec940-75b7    True
379e0b03-4ca0    True
c8bb762f-3ef9    True
c89b26ff-ec7f    True
2a793ecf-05c6    True
Name: ID, dtype: bool

#### Select and store duplicated ID's in a new varibale called dupe_ids

In [15]:
dupe_ids = id_counts.index[dupe_mask]

# the set was 30k and we had 29687 unique values with the difference being 313.
len(dupe_ids)

313

In [16]:
# lets view the dups in our data frame by using our new varibales to locate them (slicing only 3 for now to view)
df.loc[df['ID'].isin(dupe_ids[0:3]), :]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
18229,c8bb762f-3ef9,50000,1,2,1,54,Not available,2,2,3,...,38233,37318,39757,0,5500,0,0,3036,2000,1
18329,c8bb762f-3ef9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19066,379e0b03-4ca0,40000,2,2,1,41,0,0,0,0,...,2424,822,0,1176,1049,1000,0,0,0,0
19166,379e0b03-4ca0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20648,87dec940-75b7,10000,1,3,2,23,0,0,0,0,...,9414,5354,5275,1299,1337,1081,191,375,0,0
20748,87dec940-75b7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# the above data frame prodcues one row with valid data and one with all 0's but the same account number.
# Create a boolean mask for 0's and get rid of those rows.
df_zero_mask = df == 0

In [18]:
# create a feature zero mask to use the above var to find all cols with 0's 
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)

In [19]:
# 315 row have zeros for every column but the first one. This is greater than the 313 so if we delete all zero rows,
# we may get rid of the dup id problem.
sum(feature_zero_mask)

315

In [21]:
# clean the data frame by eliminating the rows with all zeros excpet for ID
# note the use of the ~ as a logical NOT operator to select rows with no zeros and : to select all cols.
# the goal is to see if the remaining rows are equal to the unique id's
df_clean1 = df.loc[~feature_zero_mask, :].copy()

In [22]:
# test the total of unique id's
df_clean1.shape

(29685, 25)

In [24]:
# it is a match
df_clean1['ID'].nunique()

29685

In [25]:
# verify each row is not null
df_clean1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29685 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          29685 non-null  object
 1   LIMIT_BAL                   29685 non-null  int64 
 2   SEX                         29685 non-null  int64 
 3   EDUCATION                   29685 non-null  int64 
 4   MARRIAGE                    29685 non-null  int64 
 5   AGE                         29685 non-null  int64 
 6   PAY_1                       29685 non-null  object
 7   PAY_2                       29685 non-null  int64 
 8   PAY_3                       29685 non-null  int64 
 9   PAY_4                       29685 non-null  int64 
 10  PAY_5                       29685 non-null  int64 
 11  PAY_6                       29685 non-null  int64 
 12  BILL_AMT1                   29685 non-null  int64 
 13  BILL_AMT2                   29685 non-null  in